In [11]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1e777777
MAX_32BIT = 0xffffffff

In [12]:
class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    # 特殊メソッドの__setitem__を利用し、setattrメソッドで属性を追加
    def __setitem__(self, key, value):
        # pythonでは.(ドット)を使うことで直接オブジェクトにアクセスすることができるが、setattrを利用すると、どのオブジェクトにどのようなkeyとvalueをつけるか指定できる
        setattr(self, key, value)

    def to_json(self):
        return {
            "index": self.index,
            "prev_hash": self.prev_hash,
            "stored_data": self.data,
            "timestamp": self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            # rjust(8, "0")は全部で指定された文字になるように右寄せにするメソッド。この場合、全部で８文字になるように右寄せし、左側の足りない部分は０で埋める処理をする。
            "bits" : hex(self.bits)[2:].rjust(8, "0"),
            "nonce" : hex(self.nonce)[2:],
            "elapsed_time": self.elapsed_time,
            "block_hash": self.block_hash
        }

    # ブロックヘッダを構築したのち、それをSHA256でハッシュ化し、その結果を返す処理
    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + str(self.bits)[2:] + str(self.nonce)# [2:]で先頭２文字を飛ばしているのは、16真数を表す0xが先頭についているため。0xを飛ばしてbitsの値そのものを抜き出すためにソートをしている。
        # hashlibは文字列をエンコードしたものを引数として渡す必要があるため、上の行のようみ全てを文字列に変換したものを.encode()によってエンコードしている。
        h = hashlib.sha256(blockheader.encode()).hexdigest()# .encode()は引数を設定しなければでジョルとでUTF-8が使われる。
        # 冒頭で定義したブロックハッシュの変数に上の行で計算したハッシュ値を代入
        self.block_hash = h
        return h

    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits

    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

In [13]:
class BlockChain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)

    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))

    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = \
                    str((end_time - start_time) / \
                        1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)

    def add_newblock(self, i):
        last_block = self.chain[-1]
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = BlockChain(INITIAL_BITS)
    print("ジェネシスブロックを作成中 ・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中 ・・・
{
  "bits": "1e777777",
  "block_hash": "0000539a8b6f8896253a4c6334c69739004ae92b5e1acb643297e24fe2e03d9c",
  "elapsed_time": "0.307秒",
  "index": 0,
  "nonce": "c2c9",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2022/06/12 22:24:44"
}
2番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00002ef6b1622e2c418c2a84a4eefa0644b1f72df38081fee8ea22919b596e3a",
  "elapsed_time": "1.209秒",
  "index": 1,
  "nonce": "34f3c",
  "prev_hash": "0000539a8b6f8896253a4c6334c69739004ae92b5e1acb643297e24fe2e03d9c",
  "stored_data": "ブロック 1",
  "timestamp": "2022/06/12 22:24:44"
}
3番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "000051b3e42a5df077cce29607a3e13a04a8a9d53c04c518ff158339f9b09eb0",
  "elapsed_time": "1.675秒",
  "index": 2,
  "nonce": "48c54",
  "prev_hash": "00002ef6b1622e2c418c2a84a4eefa0644b1f72df38081fee8ea22919b596e3a",
  "stored_data": "ブロック 2",
  "timestamp": "2022/06/12 22: